In [ ]:
%pip install -qU langchain langchain-ollama pypdf

In [1]:
import os
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [ ]:
# Load a PDF file
pdf_loader = PyPDFLoader("../00-example_data/layout-parser-paper.pdf")  # Replace with your actual file path
pdf_docs = pdf_loader.load()
print(f"Loaded {len(pdf_docs)} pages from the file.")

# Split the text into chunks for better processing
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(pdf_docs)
print(f"Total chunks created: {len(documents)}")


In [ ]:
from langchain_core.runnables import RunnableLambda

# Define a function to retrieve relevant chunks
def retrieve_documents(query):
    return "\n".join([doc.page_content for doc in documents[:3]])  # Returning top 3 chunks

# Convert the function into a Runnable
retriever_runnable = RunnableLambda(retrieve_documents)

# Define a prompt template
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Define the Ollama model
model = ChatOllama(model="llama3.2", temperature=0.1)

# Create a parallel chain that retrieves context and passes through the question
retrieval_chain = (
    {"context": retriever_runnable, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Invoke the chain
result = retrieval_chain.invoke("What is discussed in this document?")
print(result)


In [ ]:
prompt

In [ ]:
from operator import itemgetter

# Update the template to support multi-language responses
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

# Create a new chain with itemgetter for structured input extraction
chain = (
    {
        "context": itemgetter("question") | retriever_runnable,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

# Invoke the chain with a language parameter
result = chain.invoke({"question": "What is discussed in this document?", "language": "French"})
print(result)


In [ ]:
template

In [ ]:
prompt

In [ ]:
# Define multiple prompts
joke_prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
poem_prompt = ChatPromptTemplate.from_template("Write a short poem about {topic}")

model1 = ChatOllama(model="llama3.2:latest", temperature=0.1)
model2 = ChatOllama(model="gemma2:2b", temperature=0.1)


# Create separate chains for each prompt
joke_chain = joke_prompt | model1
poem_chain = poem_prompt | model2

# Run them in parallel
parallel_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

# Invoke the chain
result = parallel_chain.invoke({"topic": "stars"})
print(result)


In [ ]:
import time

# Measure single execution
start = time.time()
joke_chain.invoke({"topic": "stars"})
end = time.time()
print(f"Joke chain execution time: {end - start:.2f} seconds")

# Measure single execution
start = time.time()
poem_chain.invoke({"topic": "stars"})
end = time.time()
print(f"Poem chain execution time: {end - start:.2f} seconds")

# Measure parallel execution
start = time.time()
parallel_chain.invoke({"topic": "stars"})
end = time.time()
print(f"Parallel execution time: {end - start:.2f} seconds")
